# example

## 将几何结构单独写成类的 example
+ advanced/hadrontherapy
+ advanced/fastAerosol

# G4PVPlacement

## 构造函数 
```
G4PVPlacement(G4RotationMatrix* pRot,
          const G4ThreeVector& tlate,
          G4LogicalVolume* pCurrentLogical,
          const G4String& pName,
          G4LogicalVolume* pMotherLogical,
          G4bool pMany,
          G4int  pCopyNo,
          G4bool pSurfChk = false);
```
```
G4PVPlacement(const G4Transform3D& Transform3D,
          G4LogicalVolume* pCurrentLogical,
          const G4String& pName,
          G4LogicalVolume* pMotherLogical,
          G4bool pMany,
          G4int pCopyNo,
          G4bool pSurfChk = false);
```
+ 其中第一种构造函数自身坐标系跟随物体旋转，第二种放置，自身坐标系不跟随物体旋转。

## Example

### 基础补充
+ 矩阵可以用于表示旋转，比如绕 x 轴旋转 $\phi$ 角，表示为矩阵则为
  $$
  \left[
  \begin{matrix}
   1 & 0 & 0 \\
   0 & cos(\phi) & -sin(\phi) \\
   0 & sin(\phi) & cos(\phi)
  \end{matrix}
  \right]
  $$
  如果是多次旋转，则在原来的基础上左乘旋转矩阵，得到最终的变换矩阵。

### 示例来自 extended/geometry/transforms
+ 在该示例中，放置的是一个棱台，放置前默认的位置在原点。使用 G4Transform3D 将其旋转平移至目标位置，有两种方法，分别如下：
    
    + PlaceWithDirectMatrix()，这里用的是第二种构造函数。在该方法中，直接给出了旋转矩阵G4RotationMatrix(u, v, w)，该旋转矩阵可以通过G4RotationMatrix()单位矩阵先 rotationY(90.\*deg)，再 rotationZ(30.\*deg)得到，结果与G4RotationMatrix(u, v, w)是一致的。position1则是决定棱台中心平移后的位置。
 
    ```
    G4double phi = 30*deg;
    // u, v, w are the daughter axes, projected on the mother frame
    G4ThreeVector u = G4ThreeVector(0, 0, -1);
    G4ThreeVector v = G4ThreeVector(-std::sin(phi), std::cos(phi),0.);
    G4ThreeVector w = G4ThreeVector( std::cos(phi), std::sin(phi),0.);
    G4RotationMatrix rotm1  = G4RotationMatrix(u, v, w);
    G4cout << "\n --> phi = " << phi/deg << " deg;  direct rotation matrix : ";
    rotm1.print(G4cout);
    G4ThreeVector position1 = og*w;
    G4Transform3D transform1 = G4Transform3D(rotm1,position1);

    new G4PVPlacement(transform1,         //position, rotation
                    fTrdVolume,         //logical volume
                    "Trd",              //name
                    fWorldVolume,       //mother volume
                    false,              //no boolean operation
                    1);                 //copy number
    ```
    + PlaceWithInverseMatrix()，这里用的是第一种构造函数。相比于第一种方法，唯一的变化就是将rotm1取逆。
    ```
    G4double phi = 30*deg;
    // u, v, w are the daughter axes, projected on the mother frame
    G4ThreeVector u = G4ThreeVector(0, 0, -1);
    G4ThreeVector v = G4ThreeVector(-std::sin(phi), std::cos(phi),0.);
    G4ThreeVector w = G4ThreeVector( std::cos(phi), std::sin(phi),0.);
    G4RotationMatrix rotm1 = G4RotationMatrix(u, v, w);
    G4RotationMatrix* rotm1Inv = new G4RotationMatrix(rotm1.inverse());
    G4cout << "\n --> phi = " << phi/deg << " deg;  inverse rotation matrix : ";
    rotm1Inv->print(G4cout);
    G4ThreeVector position1 = og*w;

    new G4PVPlacement(rotm1Inv,
                    position1,
                    fTrdVolume,         //logical volume
                    "Trd",              //name
                    fWorldVolume,       //mother volume
                    false,              //no boolean operation
                    1);                 //copy number
    ```
 
 + 最终的放置位置
 <img src="./fig/20240227_0.png" alt="" width="300" height="200"> 
